<a href="https://colab.research.google.com/github/abdulmoid7416/Plant-Disease-Classification/blob/main/PlantDiseaseClassificationRESNET50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu


     |████████████████████████████████| 454.3MB 21kB/s 


In [ ]:
!nvidia-smi


Sat Jun 26 17:50:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install h5py

In [ ]:
# import the libraries

import numpy as np
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Assign image re-size setting
IMAGE_SIZE = [224, 224]

#Dataset stored in Drive
train_path = '/content/gdrive/MyDrive/PlantDiseases Dataset /Train Dataset'
valid_path = '/content/gdrive/MyDrive/PlantDiseases Dataset /Test Dataset'

In [ ]:
# Import Resnet50 model with imageneet weights, and assign IMAGE_SIZE as input_shape
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights; We will directly use the pre-trained weights from the ResNet, 
#So we dont have to train it, instead we will just train the last layers
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/gdrive/MyDrive/PlantDiseases Dataset /Train Dataset/*')

In [ ]:
# Adding our own layers - 
x = Flatten()(resnet.output)

In [ ]:
# Final layer
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset, Also Augmenting Training Data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
import os
from keras.callbacks import ModelCheckpoint


In [ ]:
# Provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/PlantDiseases Dataset /Train Dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 70653 images belonging to 38 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/PlantDiseases Dataset /Test Dataset',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 17612 images belonging to 38 classes.


In [ ]:
import os
#from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import files

In [ ]:
new_model = load_model('/content/gdrive/MyDrive/model_resnet50_update1.h5')


In [ ]:
r=new_model.fit(
  training_set,
  validation_data=test_set,
  epochs=6,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/6
  20/2208 [..............................] - ETA: 6:05:15 - loss: 20.7976 - accuracy: 0.0156

KeyboardInterrupt: ignored

In [ ]:
# fit the model
f = model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

model.save('model_resnet50.h5')

files.download("model_resnet50.h5")

Epoch 1/10
   5/2208 [..............................] - ETA: 10:15:51 - loss: 17.4893 - accuracy: 0.0312

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# save it as a h5 file

from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [ ]:
# plot the loss
plt.plot(f.history['loss'], label='train loss')
plt.plot(f.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(f.history['accuracy'], label='train acc')
plt.plot(f.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


In [ ]:
# Download the model
from google.colab import files
files.download("model_resnet50.h5")